In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [22]:
!cd ../ && cd input && cd skin-cancer9-classesisic && cd 'Skin cancer ISIC The International Skin Imaging Collaboration' && cd Train && ls

'actinic keratosis'	 melanoma		      'seborrheic keratosis'
'basal cell carcinoma'	 nevus			      'squamous cell carcinoma'
 dermatofibroma		'pigmented benign keratosis'  'vascular lesion'


## Imports
- torch: This is the main PyTorch library used for building and training models.
- torch.nn: Contains modules for creating neural networks.
- torch.optim: Contains optimizers like SGD, Adam, etc., to update model weights during training.
- torchvision: Helps with image data manipulation and model loading.
- datasets: Provides utilities to load image datasets.
- models: Contains pre-trained models that we can use (e.g., ResNet, VGG).
- transforms: Used for preprocessing the images (like resizing, normalizing, etc.).
- DataLoader: Used to load batches of data in a fast, efficient manner.
- os: Helps navigate directories.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import random_split, DataLoader
import os

## Data Transformations(for both Training and Testing Images) and Augmentation(Only for Training Images)

In [14]:
data_transforms = {
    # Training Transformations
    'train': transforms.Compose([
        transforms.Resize(256), # Resize the shorter side of image to 256 Pixels while preserving the Aspect Ratio
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # Randomly crop the image to 224*224, ensuring we keep at least 80% of the original image
        transforms.RandomHorizontalFlip(), # Randomly flip the image horizontally for Data Augmentation
        transforms.ToTensor(), # Convert Image to PyTorch Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalize the Image with Mean and Standard Deviation for each channel (ImageNet pre-trained Models expect this)
    ]),
    
    # Validation Transformation
    'val': transforms.Compose([
        transforms.Resize(256), # Resize the Shorter Side of the Image to 256 pixels
        transforms.CenterCrop(224),
        transforms.ToTensor(), # Convert Image to PyTorch Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalize the image(same normalization as in training to match the pre-trained model)
    ])
}

## Set up Datasets and Data Loaders for Training and Validation

In [15]:
# Define the directory path for training and validation datasets
data_dir = '../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration'

# Load datasets with ImageFolder, which expects data to be organized by class folders
"""
ImageFolder function automatically labels images based on their folder names (e.g., melanoma, nevus, etc.).
It loads images from the directory and applies the transformations defined earlier.
"""
train_dataset_full = datasets.ImageFolder(os.path.join(data_dir, 'Train'), data_transforms['train'])

# Split the train dataset into training and validation sets
train_ratio = 0.8
train_size = int(train_ratio * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size
train_dataset, val_dataset = random_split(train_dataset_full, [train_size, val_size])

test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Test'), data_transforms['val'])

# image_datasets = {
#     'train': datasets.ImageFolder(os.path.join(data_dir,'Train'), data_transforms['train']), # Load and pre-process the Train data
#     'val': datasets.ImageFolder(os.path.join(data_dir, 'Test'), data_transforms['val']) # # Load and pre-process the Test data
# }

# Create Dataloaders to load the data in batches of 32
dataloaders = {
    'train': DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 4), # Shuffle is done to ensure Randomness in sequence of data
    'val': DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 4),
    'test': DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 4)
}

# Get the sizes of the datasets for future reference
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}
# dataset_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']} # Returns dictionary of form {'train':1000, 'val':200}


class_names = train_dataset_full.classes # Get the class names (directories names) for label mapping

# Check if GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Print important insights
print(f"The size of Train Image = {dataset_sizes['train']}")
print(f"The size of Test Image = {dataset_sizes['val']}")
print(f"The device used => {device}")
print(f"The class names are: {class_names}")

The size of Train Image = 1791
The size of Test Image = 448
The device used => cuda:0
The class names are: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


## Initialize a Pre-Trained Model for Transfer Learning

In [20]:
model_ft = models.resnet18(weights = models.ResNet18_Weights.DEFAULT) # Load the Pre-trained ResNet18 Model
"""
ResNet18 Architecture Before: Convolutional Layers -> Global Average Pooling -> FC (512 -> 1000)
"""
num_ftrs = model_ft.fc.in_features # Get the number of input features to the final fully connected layer

model_ft.fc = nn.Linear(num_ftrs, len(class_names)) # Modify the final layer to match the number of classes (9 in our case)
"""
ResNet18 Architecture Before: Convolutional Layers -> Global Average Pooling -> FC (512 -> 9)
"""
model_ft = model_ft.to(device) # Move the model the the device (GPU if available otherwise to the CPU)

criterion = nn.CrossEntropyLoss() # Defining the Loss function (Cross Entropy Loss for multi-class Classification)

# Freeze the earlier layers and only fine-tune the fully connected layers
for param in model_ft.parameters():
#     param.requires_grad = False # Freeze all layers except the last layer
      param.requires_grad = True # Allow Updating of Convolution and Pooling layers because Freezing did not give good results

# Unfreeze the fully connected layer (the one we modified)
for param in model_ft.fc.parameters():
    param.requires_grad = True # Allow updating of the Final Layer only

optimizer_ft = optim.Adam(model_ft.parameters(), lr = 1e-4) # Setting up the Optimizer (Adam in this case)


## Training and Validating the Model

In [21]:
# Function to train the Model
def train_model(model, dataloaders, criterion, optimizer, num_epochs = 25):
    # Track the best model based on Validation Accuracy
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    # Loop over the epochs
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-"*10)
        
        # Each epoch has a training and validation loop
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # Set the Model to training mode
            else:
                model.eval() # Set the Model to Evaluation mode
            
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data batches
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad() # Clear the gradients from the previous batch to avoid accumulation.
                
                # Forward Pass: Track history if only in Train
                with torch.set_grad_enabled(phase == 'train'): # torch.set_grad_enabled(True) means keep track of Gradients
                    outputs = model(inputs) # Perform a forward pass through the model.
                    _, preds = torch.max(outputs, 1) # Returns max_value and index_of_max_value among the second dimension(1) of the (32,9) logits outputs
                    loss = criterion(outputs, labels)
                    
                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward() # Computes the gradients using backpropagation
                        optimizer.step() # Updates the model parameters based on the gradients
                
                # Track the Running loss and Number of correct predictions for each batch
                running_loss += loss.item() * inputs.size(0) # loss.item() gives the average loss over the batch in scalar form rather than Tensor(thanks to .item() function) which is then multiplied by the batch size (inputs.size(0)) to obtain the total loss in the batch
                running_corrects += torch.sum(preds == labels.data)
                
            # Calculate epoch loss and acuracy
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            
            # Deep copy the model if validation accuracy is the best
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
    
    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model


## Train the Model

In [22]:
model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs = 25)

Epoch 1/25
----------
train Loss: 1.3718 Acc: 0.5366
val Loss: 1.0352 Acc: 0.6138
Epoch 2/25
----------
train Loss: 0.7395 Acc: 0.7443
val Loss: 0.8370 Acc: 0.7143
Epoch 3/25
----------
train Loss: 0.5692 Acc: 0.7873
val Loss: 0.7951 Acc: 0.7232
Epoch 4/25
----------
train Loss: 0.4338 Acc: 0.8481
val Loss: 0.7725 Acc: 0.7031
Epoch 5/25
----------
train Loss: 0.3480 Acc: 0.8688
val Loss: 0.7370 Acc: 0.7277
Epoch 6/25
----------
train Loss: 0.2760 Acc: 0.8978
val Loss: 0.8091 Acc: 0.7143
Epoch 7/25
----------
train Loss: 0.2466 Acc: 0.9056
val Loss: 0.7976 Acc: 0.7143
Epoch 8/25
----------
train Loss: 0.2034 Acc: 0.9196
val Loss: 0.7149 Acc: 0.7366
Epoch 9/25
----------
train Loss: 0.2038 Acc: 0.9090
val Loss: 0.8637 Acc: 0.7054
Epoch 10/25
----------
train Loss: 0.2107 Acc: 0.9107
val Loss: 0.8241 Acc: 0.6920
Epoch 11/25
----------
train Loss: 0.1792 Acc: 0.9179
val Loss: 0.7720 Acc: 0.7277
Epoch 12/25
----------
train Loss: 0.1733 Acc: 0.9202
val Loss: 0.7829 Acc: 0.7321
Epoch 13/25
-

## Evaluate the Model in Test Dataset

In [23]:
def evaluate_model(model, dataloader, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    running_corrects = 0

    # Turn off gradients for faster computation
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    # Calculate total loss and accuracy
    total_loss = running_loss / len(dataloader.dataset)
    total_acc = running_corrects.double() / len(dataloader.dataset)

    print(f'Test Loss: {total_loss:.4f} Acc: {total_acc:.4f}')
    return total_loss, total_acc


In [25]:
test_loss, test_acc = evaluate_model(model_ft, dataloaders['test'], criterion)

Test Loss: 2.6205 Acc: 0.4915


## Save the Trained Model

In [26]:
torch.save(model_ft.state_dict(), 'best_model_weights.pth')

## Load the Saved Model

In [27]:
model_ft.load_state_dict(torch.load('best_model_weights.pth', weights_only = True))

model_ft = model_ft.to(device)

model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Analyze the Model Performance

In [28]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming you have your predictions and true labels stored in arrays
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Flatten the lists of predictions and labels
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

print(conf_matrix)

[[ 1  0  0  0 12  3  0  0  0]
 [ 0 14  0  0  0  2  0  0  0]
 [ 0  2  6  1  2  5  0  0  0]
 [ 0  0  0  1 11  3  0  0  1]
 [ 0  0  0  1 15  0  0  0  0]
 [ 0  1  0  0  0 15  0  0  0]
 [ 0  0  0  3  0  0  0  0  0]
 [ 0  2  0  2  1  8  0  3  0]
 [ 0  0  0  0  0  0  0  0  3]]


## Calculate accuracy, precision, recall, and F1-score

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [32]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.4915
Precision: 0.6742
Recall: 0.4915
F1-Score: 0.4239
